<a href="https://colab.research.google.com/github/jikerbug/jibaek_project_generator/blob/master/ACE_vanilla_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import sys
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import os.path


def load_data():
    input_metadata_chroma_list =[]
    output_annotations_chord_list =[]
    chord_time_segmentation = []

    path_metadata = './metadata/metadata'
    csv_metadata = '/bothchroma.csv'
    path_annotations = './annotations/annotations/'
    csv_annotations = '/majmin.lab'
    
    for folder_num in range(3,60):
        if(folder_num < 10):
            path_variable = '/000' + str(folder_num)
        elif(folder_num < 100):
            path_variable = '/00' + str(folder_num)
        elif(folder_num < 1000):
            path_variable = '/0' + str(folder_num)
        else:
            path_variable = '/' + str(folder_num)
        
        
        
        complete_path_metadata = path_metadata + path_variable + csv_metadata
        complete_path_annotations = path_annotations + path_variable + csv_annotations
        
        #파일이 존재하는지 체크
        if(os.path.isfile(complete_path_metadata)):
            pass
        else:
            continue
        
        
        
        with open(complete_path_metadata, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            flag = 0
            for segmented_metadata in reader:
                input_metadata_chroma_list.append(segmented_metadata)
                if flag == 400: # 하나의 곡에서만 너무 많은 학습을 진행시키지 않자 정확도가 급격히 올라간 모습을 볼 수 있었다....!!!무려 54였다...!
                    # 혹시 49개를 43개로 고쳤기때문일수도 있겠다
                    # 하지만 학습이 진행되도 정확도가 개선되지 않는 큰 문제가 있었다...
                    break

                flag +=1

        with open(complete_path_annotations, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            flag = 0
            for segmented_chord_data in reader:



                if(segmented_chord_data != []):
                    output_annotations_chord_list.append(segmented_chord_data[0].split('\t')[-1])
                    segmentation =[]
                    segmentation.append(segmented_chord_data[0].split('\t')[0])
                    segmentation.append(segmented_chord_data[0].split('\t')[1])

                    chord_time_segmentation.append(segmentation)
                else:
                    break
                #
                # flag +=1

    retyped_chord_to_int_list = []
    type1 = ':maj'
    type2 = ':min'
    type3 = 'b:maj'
    type4 = 'b:min'
    type5 = '#:maj'
    type6 = '#:min'
    type_table = [type1, type2, type3, type4, type5, type6]

    root_chord_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

    for chord in output_annotations_chord_list:
        escape_flag = False
        for chord_num, root_chord in enumerate(root_chord_list):
            if (escape_flag):
                break
            for type_num, type in enumerate(type_table):
                if (chord == (root_chord + type)):
                    retyped_chord_to_int_list.append(chord_num * 4 + type_num)
                    escape_flag = True
                    break
                elif (chord == 'N' or chord == 'X'):
                    retyped_chord_to_int_list.append(42)
                    escape_flag = True
                    break


    #시간 데이터 데이터 삭제
    chroma_time_index_list = []
    for chroma in input_metadata_chroma_list:
        chroma_time_index_list.append(chroma[1])
        del chroma[0]
        del chroma[0]
        #len(chroma) == 24


    # string값을 float으로 바꿔서 넣어주기
    retyped_chroma_string_to_float = []
    for chroma in input_metadata_chroma_list:
        retyped_list = []
        for value in chroma:
            temp_list = []
            temp_list.append(float(value))
            retyped_list.append(temp_list)
        retyped_chroma_string_to_float.append(retyped_list)


    print(retyped_chroma_string_to_float[0])
    print(chroma_time_index_list[0])
    print(chord_time_segmentation[0])
    print(retyped_chord_to_int_list[0])


    chroma_time_related_chord_to_int_list = []
    for chroma_time in chroma_time_index_list:
        for time, chord in zip(chord_time_segmentation, retyped_chord_to_int_list):
            start = time[0]
            end = time[1]
            #print(f'chroma_time:{chroma_time} start:{start} end:{end}')
            if(float(chroma_time) >= float(start) and float(chroma_time) < float(end)):
                chroma_time_related_chord_to_int_list.append(chord)
                break



    print("here")
    print(len(chroma_time_related_chord_to_int_list))
    print(len(retyped_chroma_string_to_float))
    print(retyped_chroma_string_to_float[0])
    print(chroma_time_related_chord_to_int_list[0])





    #n_deleted_output_annotations_chord_list = output_annotations_chord_list
    # for chroma, chord in zip(input_metadata_chroma_list, output_annotations_chord_list):
    #     print("test" + chord)
        
    #     if chord == 'N':
            
    #         n_deleted_input_metadata_chroma_list.remove(chroma)
    #         n_deleted_output_annotations_chord_list.remove(chord)
    # print(len(retyped_chroma))
    # print(len(input_metadata_chroma_list))

    for i in output_annotations_chord_list:
        print(i)

    X = np.array(retyped_chroma_string_to_float)
    y = np.array(chroma_time_related_chord_to_int_list)


    return X, y

#input_data, output_data, segmentation_data = load_data()
#print(input_data)
#print(output_data)
#print(segmentation_data)

# for segment in segmentation_data:
#     print(float(segment[1]) - float(segment[0]))



def plot_history(history):
    """Plots accuracy/loss for training/validation set as a function of the epochs
        :param history: Training history of model
        :return:
    """

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # create error sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.show()



def prepare_datasets(test_size, validation_size):


    # load data
    X, y= load_data()
    print("Look at here")
    print(len(X))
    print(len(y))

    # create train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    # create train/validation split
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # cnn과 달리 rnn에서는 이러한 3rd 차원이 필요 없다.
    # X_train = X_train[..., np.newaxis] # 3d array -> (num_samples = 130, 13, 1)
    # X_validation = X_validation[..., np.newaxis] # ... : 기존의 것들
    # X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test

def build_model(input_shape):

    # create RNN model
    model = keras.Sequential()

    # 2 LSTM layers
    model.add(keras.layers.LSTM(256, input_shape=input_shape, return_sequences=True))
    # return_sequences : second lstm에서 이 시퀀스를 사용하고 싶기 떄문에 true로 한다.
    #model.add(keras.layers.LSTM(64, input_shape=input_shape, return_sequences=True)) #레이어를 하나 더 추가하는 것은 큰 효용이 없었다.
    model.add(keras.layers.LSTM(256))

    # dense layer
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dropout(0.3))


    # output layer
    model.add(keras.layers.Dense(43, activation='softmax'))

    return model

def predict(model, X, y):

    X = X[np.newaxis, ...]

    # prediction = [[0.1, 0.2, ...]] # softmax의 결과물
    prediction = model.predict(X) # X -> (1, 130, 13, 1)

    # extract index with max value
    predicted_index = np.argmax(prediction, axis=1) # [4]
    print("Expected index: {}, Predicted index: {}".format(y, predicted_index))

if __name__ == "__main__":
    pass
    # create train, validation and test sets
    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

    print(len(y_train))

    print(y_train[0])
    print(y_test[0])
    print(y_validation[0])

    print(len(X_train))
    print(len(X_train[0]))
    print(X_train[0])
    print(X_test[0])
    print(X_validation[0])
    #sys.exit()
    # create network
    print("heoollo")
    print(X_train.shape[1],print(X_train.shape[1], ))
    input_shape = (X_train.shape[1], X_train.shape[2]) # (130, 13) (number of slices extract mfccs, mfccs)
    model = build_model(input_shape)

    # compile model
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer,
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])

    model.summary()

    # train model
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

    #plot accuracy/error for training and validation
    plot_history(history)

    # evaluate model on test set
    test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
    print("Accuracy on test set is: {}".format(test_accuracy))

    # make prediction on a sample
    X = X_test[100]
    y = y_test[100]


    predict(model, X, y)


